<a href="https://colab.research.google.com/github/creamburger/TensorStudy/blob/main/%EB%8C%80%EC%B6%9C_%EC%9C%84%ED%97%98%EB%8F%84_%EC%98%88%EC%B8%A1_%EB%AA%A8%EB%8D%B8_ipynb%EC%9D%98_%EC%82%AC%EB%B3%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# GPU를 사용하기 위해 google colab 사용해서 file import.

from google.colab import files
myfile = files.upload()

Saving cs_data.csv to cs_data (1).csv


In [3]:
#io와 pandas 모듈 import
import io
import pandas as pd

In [4]:
#pd.read_csv로 csv파일 불러오기
col=['no','SeriousDlqin2yrs','RevolvingUtilizationOfUnsecuredLines','age','NumberOfTime30-59DaysPastDueNotWorse','DebtRatio','MonthlyIncome','NumberOfOpenCreditLinesAndLoans','NumberOfTimes90DaysLate','NumberRealEstateLoansOrLines','NumberOfTime60-89DaysPastDueNotWorse','NumberOfDependents']
data = pd.read_csv(io.BytesIO(myfile['cs_data.csv']),
                   header=None,   #첫번째 행이 데이터이므로 header가 없다라고 설정.
                  names=col,
                  skipinitialspace=True, # , 다음의 공백을 제거,
                   na_values='?')
data.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,no,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
0,NaN,SeriousDlqin2yrs,RevolvingUtilizationOfUnsecuredLines,age,NumberOfTime30-59DaysPastDueNotWorse,DebtRatio,MonthlyIncome,NumberOfOpenCreditLinesAndLoans,NumberOfTimes90DaysLate,NumberRealEstateLoansOrLines,NumberOfTime60-89DaysPastDueNotWorse,NumberOfDependents
1,1.0,1,0.766126609,45,2,0.802982129,9120,13,0,6,0,2
2,2.0,0,0.957151019,40,0,0.121876201,2600,4,0,0,0,1
3,3.0,0,0.65818014,38,1,0.085113375,3042,2,1,0,0,0
4,4.0,0,0.233809776,30,0,0.036049682,3300,5,0,0,0,0


In [5]:
# col을 지정해줬으므로 첫번째 인덱스를 drop하고 no라는 인덱스를 지정
df = data
df = df.drop([0], axis = 0)
df = df.set_index('no')

In [6]:
# 결측치 제거
df.dropna(inplace=True)

In [7]:
# 형변환
df = df.astype('float')

In [8]:
# X(SeriousDlqin2yrs을 제외한 feature들), y(SeriousDlqin2yrs) 분리
y = df['SeriousDlqin2yrs']
X = df.drop(columns='SeriousDlqin2yrs')
print(type(X), type(y))
X.columns, y.name

<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>


(Index(['RevolvingUtilizationOfUnsecuredLines', 'age',
        'NumberOfTime30-59DaysPastDueNotWorse', 'DebtRatio', 'MonthlyIncome',
        'NumberOfOpenCreditLinesAndLoans', 'NumberOfTimes90DaysLate',
        'NumberRealEstateLoansOrLines', 'NumberOfTime60-89DaysPastDueNotWorse',
        'NumberOfDependents'],
       dtype='object'), 'SeriousDlqin2yrs')

In [9]:
# 테스트 셋 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)  #test set 분리
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=0) # train/val set 분리

print(X_train.shape, X_test.shape)

(96215, 10) (24054, 10)


In [10]:
# metrics 함수 직접 불러오기


from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.metrics import precision_recall_curve, average_precision_score, PrecisionRecallDisplay, roc_curve, roc_auc_score, RocCurveDisplay
import matplotlib.pyplot as plt

# 분류문제 평가 함수
def print_metrics_classification(y, pred, title=None):
   """
   정답(target)과 모델이 추론한 값을 받아 정확도, 재현율, 정밀도, f1 점수를 출력
   [parameter]
       y: ndarray - 정답(target)
       pred: ndarray - 모델이 추론한 결과
       title: str - 출력할 내용의 title. 기본값: None = 출력안한다.
   [return value]
   [exception]
   """
   if title:
       print(title)
   print('정확도(accuracy):', accuracy_score(y, pred))
   print('재현율(recall):', recall_score(y, pred))
   print('정밀도(precision)', precision_score(y, pred))
   print('F1점수:', f1_score(y, pred))

def plot_confusionmatrix(y, pred, title=None):
   """
   정답과 모델이 추론한 결과를 이용해 Confusion Matrix를 시각화(히트맵)하는 함수
   [parameter]
       y: ndarray - 정답(target)
       pred: ndarray - 모델이 추론한 결과
       title: str - 출력할 내용의 title. 기본값: None = 출력안한다.
   """
   cm = confusion_matrix(y, pred)
   disp = ConfusionMatrixDisplay(cm)
   disp.plot(cmap='Blues')
   if title:
       plt.title(title)
   plt.show()

   
def plot_precisionrecall_curve(y, pos_proba, title=None):
   """
   Precision Recall Curve 를 시각화하는 함수
   [parameter]
       y: ndarray - 정답
       pos_proba: ndarray - positive(양성)의 확률
       title: str - 출력할 내용의 title
   """
   ap_score = average_precision_score(y, pos_proba)
   precisions, recalls, threshs = precision_recall_curve(y, pos_proba)
   disp = PrecisionRecallDisplay(precisions, recalls, average_precision=ap_score)
   disp.plot()
   if title:
       plt.title(title)
   plt.show()
   
def plot_roccurve(y, pos_proba, title=None):
   """
   ROC Curve를 시각화하는 함수
   [parameter]
       y: ndarry - 정답
       pos_proba: ndarray - positive(양성)의 확률
       title: str - 출력할 내용의 title
   """
   auc_score = roc_auc_score(y, pos_proba)
   fprs, tprs, threshs = roc_curve(y, pos_proba)
   disp = RocCurveDisplay(fpr=fprs, tpr=tprs, roc_auc=auc_score)
   disp.plot()
   if title:
       plt.title(title)
   plt.show()


In [11]:
# 사용할 모델들 import
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import  GridSearchCV
from sklearn.pipeline import Pipeline

# 모델들
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier


from sklearn.metrics import roc_auc_score

In [12]:
# 스케일러
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:
# Pipeline을 통해 대략적인 모델들의 성능 평가. 여기서 가장 좋은 모델을 뽑아 추후 하이퍼파라미터 튜닝할 것.
svm = Pipeline([('scaler', StandardScaler()), 
                ("svm", SVC(random_state=0))], 
              verbose=True)
knn = Pipeline([('scaler', StandardScaler()),
                ('knn', KNeighborsClassifier(n_neighbors=5))
               ], verbose=True)
lr = Pipeline([('scaler', StandardScaler()),
               ('lr', LogisticRegression(max_iter=1000, random_state=0))])

rfc = RandomForestClassifier(n_estimators=200, max_depth=4, random_state=0)

xgb = XGBClassifier(n_estimators=200, max_depth=2, random_state=0)

In [14]:
estimators = [('svm', svm),  #(모델이름, 모델객체)
              ('knn', knn), 
              ('LogisticRegression', lr), 
              ('Random Forest', rfc),
              ('XGBoost', xgb)]

# 각 모델의 test set 추론 결과를 저장할 딕셔너리
pred_test_dict = {}

for model_name, model  in estimators:
    # 학습
    model.fit(X_train_scaled, y_train)
    # 추론
    pred_train = model.predict(X_train_scaled)
    pred_test = model.predict(X_test_scaled)
    
    pred_test_dict[model_name] = pred_test
    
    # 평가
    print(model_name)
    print(roc_auc_score(y_train, pred_train))
    print_metrics_classification(y_train,pred_train,"Trainset 평가")
    print("=============")
    print_metrics_classification(y_test,pred_test,"testset 평가")
    print("=============")
#     print_metrics_classification(y_train, pred_train, f'{model_name}의 Train set 평가')
#     print('-------------------------')
#     print_metrics_classification(y_test, pred_test, f'{model_name}의 Test set 평가')
#     print("================================================")

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total= 9.1min
svm
0.5260507003302326
Trainset 평가
정확도(accuracy): 0.9326300472899236
재현율(recall): 0.05384385282680227
정밀도(precision) 0.6976744186046512
F1점수: 0.09997222993612884
testset 평가
정확도(accuracy): 0.9336908622266566
재현율(recall): 0.06403351286654697
정밀도(precision) 0.7753623188405797
F1점수: 0.11829740187949142
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.2s
knn
0.6106465428456718
Trainset 평가
정확도(accuracy): 0.9401652548978849
재현율(recall): 0.2279389769667963
정밀도(precision) 0.7192071731949032
F1점수: 0.3461669505962521
testset 평가
정확도(accuracy): 0.9291178182422882
재현율(recall): 0.13764213046080193
정밀도(precision) 0.46558704453441296
F1점수: 0.21247113163972287
LogisticRegression
0.5162137940908937
Trainset 평가
정확도(accuracy): 0.9309359247518578
재현율(recall): 0.03454980556386479


놀랍게도 가장 기본 모델인 knn이 평가지표에서 0.6106465428456718로 1위를 했다.
 - 단순한 모델일수록 더 잘 예측하는 게 아닐까?
 - 과적합이 많이 일어나고 있는 게 아닐까?

In [25]:
from sklearn.ensemble import GradientBoostingClassifier #RandomForestClassifier, AdaBoostClassifier

그럼 앙상블 모델인 GradientBoostingClassifier를 사용해서 앙상블의 성능을 보자.

In [29]:

gbc = GradientBoostingClassifier(random_state=0)
param_grid = {'n_estimators' : [20,30,40], "max_depth": [2,5,7,9],
"max_features": [2,3,5] ,'learning_rate' : [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
}
# RandomizedSearchCV로 최적의 파라미터 찾기(GridSearchCV 너무오래 걸려서 포기...)
grid = RandomizedSearchCV(gbc, param_grid, cv=5,scoring='roc_auc',n_iter = 10)
grid.fit(X_train_scaled, y_train)
print("Grid-Search with roc_auc")
print("Best parameters:", grid.best_params_)
print("Best cross-validation score (auc_roc)): {:.3f}".format(grid.best_score_))
print("====================================")
print("Test set roc_auc_score: {:.3f}".format(
roc_auc_score(y_test, grid.decision_function(X_test_scaled)))) #decision_function: 찾았던 최고의 파라미터로 해줌 : Call decision_function on the estimator with the best found parameters.

Grid-Search with roc_auc
Best parameters: {'n_estimators': 40, 'max_features': 2, 'max_depth': 7, 'learning_rate': 0.1}
Best cross-validation score (auc_roc)): 0.854
Test set AUC: 0.855


0.855로 목표치였던 0.85를 넘김. 과적합도 안 일어남!
> 다만 모델이 단순할수록 성능이 괜찮았던 걸 보아 영향을 주지 않았던 컬럼들을 드롭하고 다시 계산하면 성능이 더 좋아지지 않을까 고민됨.




In [42]:
import pandas as pd
gb2 = GradientBoostingClassifier(random_state=0, n_estimators=40, max_features=2, max_depth=7, learning_rate=0.1)
gb2.fit(X_train_scaled, y_train)
fi = pd.Series(gb2.feature_importances_)
fi.sort_values(ascending=True)

9    0.015547
7    0.021293
5    0.034490
1    0.043032
4    0.045900
3    0.059233
2    0.124361
8    0.160634
0    0.195361
6    0.300149
dtype: float64

10, 8번 컬럼이 안 중요하다는 것을 알게 됨(X에는 0번 인덱스가 빠졌으므로 원래 dataFrame에서 컬럼번호+1 해줘야 구분가능함)
- 스케일링을 해둬서 keyvalue 오류가 생김->인덱스 번호로 구분해야 함....
- NumberOfDependents 
- NumberRealEstateLoansOrLines 컬럼을 제외해보자.

In [41]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Float64Index: 120269 entries, 1.0 to 150000.0
Data columns (total 11 columns):
 #   Column                                Non-Null Count   Dtype  
---  ------                                --------------   -----  
 0   SeriousDlqin2yrs                      120269 non-null  float64
 1   RevolvingUtilizationOfUnsecuredLines  120269 non-null  float64
 2   age                                   120269 non-null  float64
 3   NumberOfTime30-59DaysPastDueNotWorse  120269 non-null  float64
 4   DebtRatio                             120269 non-null  float64
 5   MonthlyIncome                         120269 non-null  float64
 6   NumberOfOpenCreditLinesAndLoans       120269 non-null  float64
 7   NumberOfTimes90DaysLate               120269 non-null  float64
 8   NumberRealEstateLoansOrLines          120269 non-null  float64
 9   NumberOfTime60-89DaysPastDueNotWorse  120269 non-null  float64
 10  NumberOfDependents                    120269 non-null  float64

In [43]:
X = df.drop(columns=['SeriousDlqin2yrs','NumberOfDependents','NumberRealEstateLoansOrLines' ])

In [44]:
# 테스트 셋 분할
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=0)  #test set 분리
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, stratify=y_train, random_state=0) # train/val set 분리

print(X_train.shape, X_test.shape)

(96215, 8) (24054, 8)


In [45]:
# 스케일러
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [46]:
gbc = GradientBoostingClassifier(random_state=0)
param_grid = {'n_estimators' : [20,30,40], "max_depth": [2,5,7,9],
"max_features": [2,3,5] ,'learning_rate' : [0.05, 0.075, 0.1, 0.25, 0.5, 0.75, 1]
}
# RandomizedSearchCV로 최적의 파라미터 찾기(GridSearchCV 너무오래 걸려서 포기...)
grid = RandomizedSearchCV(gbc, param_grid, cv=5,scoring='roc_auc',n_iter = 10)
grid.fit(X_train_scaled, y_train)
print("Grid-Search with roc_auc")
print("Best parameters:", grid.best_params_)
print("Best cross-validation score (auc_roc)): {:.3f}".format(grid.best_score_))
print("====================================")
print("Test set roc_auc_score: {:.3f}".format(
roc_auc_score(y_test, grid.decision_function(X_test_scaled))))

Grid-Search with roc_auc
Best parameters: {'n_estimators': 40, 'max_features': 2, 'max_depth': 7, 'learning_rate': 0.05}
Best cross-validation score (auc_roc)): 0.853
Test set roc_auc_score: 0.853


역시나 예상대로 과적합이 일어나지도 않았는데 feature를 줄이는 것은 별 소용없다는 것을 증명했다.
> 하지만 단일 모델들은 어떨까?

In [48]:
for model_name, model  in estimators:
    # 학습
    model.fit(X_train_scaled, y_train)
    # 추론
    pred_train = model.predict(X_train_scaled)
    pred_test = model.predict(X_test_scaled)
    
    pred_test_dict[model_name] = pred_test
    
    # 평가
    print(model_name)
    print(roc_auc_score(y_train, pred_train))
    print_metrics_classification(y_train,pred_train,"Trainset 평가")
    print("=============")
    print_metrics_classification(y_test,pred_test,"testset 평가")
    print("*************************************")

[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing svm, total= 3.7min
svm
0.5307406564056418
Trainset 평가
정확도(accuracy): 0.9328587018656135
재현율(recall): 0.06371522584504935
정밀도(precision) 0.6805111821086262
F1점수: 0.11652078774617067
testset 평가
정확도(accuracy): 0.9338987278623098
재현율(recall): 0.0718132854578097
정밀도(precision) 0.7547169811320755
F1점수: 0.13114754098360656
*************************************
[Pipeline] ............ (step 1 of 2) Processing scaler, total=   0.0s
[Pipeline] ............... (step 2 of 2) Processing knn, total=   0.1s
knn
0.6304691272166262
Trainset 평가
정확도(accuracy): 0.941641116250065
재현율(recall): 0.26906969787615914
정밀도(precision) 0.7119113573407202
F1점수: 0.3905351134266797
testset 평가
정확도(accuracy): 0.9282032094454145
재현율(recall): 0.17235188509874327
정밀도(precision) 0.45569620253164556
F1점수: 0.25010855405992183
*************************************
LogisticRegression
0.5160809821807948
Trai

오르기는 올랐지만 별로 소용 없었던 걸로.....

In [19]:
from sklearn.model_selection import RandomizedSearchCV

params = {
    'max_depth':range(2,15,1), 
    "min_samples_split":range(2,21,2),
    "min_samples_leaf":[0.01, 0.02, 0.03,0.04, 0.05], 
    "max_leaf_nodes": range(10, 71, 5),
    "max_features":['auto', 0.95, 0.9, 0.85, 0.8, 0.75, 0.7],
    "splitter":['random', 'best']
}

In [20]:
rs = RandomizedSearchCV(DecisionTreeClassifier(random_state=0), 
                        params, 
                        n_iter=100,  # 몇개의 조합만 테스트 할 것인지 개수.
                        scoring='roc_auc', 
                        cv=5,
                        n_jobs=-1
                       )
rs.fit(X_train_scaled, y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(random_state=0),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': range(2, 15),
                                        'max_features': ['auto', 0.95, 0.9,
                                                         0.85, 0.8, 0.75, 0.7],
                                        'max_leaf_nodes': range(10, 71, 5),
                                        'min_samples_leaf': [0.01, 0.02, 0.03,
                                                             0.04, 0.05],
                                        'min_samples_split': range(2, 21, 2),
                                        'splitter': ['random', 'best']},
                   scoring='roc_auc')

In [21]:
#roc_auc기반
from sklearn.metrics import average_precision_score, roc_auc_score

grid3 = GridSearchCV(DecisionTreeClassifier(random_state=0),param_grid=params, scoring ="roc_auc")
grid3.fit(X_train_scaled,y_train)
y_pred33 = grid3.decision_function(X_test)
print(grid3.best_params_)
print(grid3.best_score_)
print(grid3.score(X_test,y_test)) #해당것 score
print(roc_auc_score(y_test,y_pred33)) #roc score

print("얘는 NG")
y_pred3 = grid3.predict(X_test) #테스트 데이터로 예측값 받음
print(roc_auc_score(y_test,y_pred3)) #roc score

KeyboardInterrupt: ignored